<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Import-&amp;-Load-data" data-toc-modified-id="Import-&amp;-Load-data-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Import &amp; Load data</a></span></li></ul></li><li><span><a href="#Configuration" data-toc-modified-id="Configuration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Configuration</a></span></li><li><span><a href="#Optuna" data-toc-modified-id="Optuna-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Optuna</a></span></li><li><span><a href="#lgb-model" data-toc-modified-id="lgb-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>lgb model</a></span></li></ul></div>

# TMDB_Hyperparameter Tuning by using Optuna

### Introduction

In this notebook, I will tune hyperparamters of lightGBM model by using Optuna. 

### Import & Load data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

import optuna

pd.set_option('display.max_columns', 100)

from pathlib import Path
import os
os.getcwd()

'/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/TMDB'

In [2]:
path = Path('/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/TMDB/datasets/')

In [3]:
import sys
import time
import random
import logging
import typing as tp
from contextlib import contextmanager

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error, mean_squared_error

import lightgbm as lgb
from lightgbm import LGBMRegressor

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# from feature-engine
from feature_engine.imputation import (
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer
)


from feature_engine.encoding import (
    OrdinalEncoder
)

from feature_engine.transformation import LogTransformer

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper

import preprocessors as pp

from math import sqrt

In [5]:
df_train = pd.read_csv(path/'engineered_train.csv')
df_test = pd.read_csv(path/'engineered_test.csv')
df_train.shape, df_test.shape

((3000, 19), (4398, 18))

## Configuration

In [14]:
# categorical variables with NA with set
CATEGORICAL_VARS_WITH_NA_FREQUENT = ['num_genres', 'num_spoken_lang']

CATEGORICAL_VARS_WITH_NA_MISSING = ['num_pro_comps', 'num_keywords']


# numerical variables with NA in train set
NUMERICAL_VARS_WITH_NA = ['budget',
                          'runtime',
                          'num_cast']

# this is for test set
NEW_VARS_WITH_NA = ['release_year', 'release_month', 'release_day', 'release_dayofweek']

# temporal variable and binary variables
TEMPORAL_VARS = ['release_year', 'release_month', 'release_day', 'release_dayofweek']
BINARY_VARS = ['is_collection', 'is_homepage', 'is_English']

# variables to log transform
NUMERICAL_LOG_VARS = ['budget', 'popularity', 'num_cast']

# variables to map
genre = ['num_genres']
comps = ['num_pro_comps']
count = ['num_pro_count']
lang = ['num_spoken_lang']
keywords = ['num_keywords']
crew = ['num_crew']

# variables to map
genre_mappings = {'1': 1, '2': 2, '3': 3, '4': 4, '>4': 5}
comps_mappings = {'1': 1, '2': 2, '3':3, '4': 4, '5': 5, '>5': 6, 'Missing': 0}
count_mappings = {'1': 1, '2': 2, '>2': 3}
lang_mappings = {'1': 1, '2': 2, '3': 3, '>3': 4}
keywords_mappings = {'<5': 1, '<10': 2, '<15': 3, '<20': 4, '>30': 5, 'Missing': 0}
crew_mappings = {'small': 1, 'medium': 2, 'large': 3, 'missing': 0}

## Optuna

In [30]:
def objective(trial): 
    # set up hyperparameters to figure out best parameters
    params = {
            'n_estimators': trial.suggest_categorical('n_estimators', [5000, 10000]),
            'num_leaves': trial.suggest_int('num_leaves', 2, 2**4), #2**11),
            'boosting_type': trial.suggest_categorical('boosting', ['gbdt']),
            'max_depth': trial.suggest_int('max_depth', 2, 2**4),
            'max_bin': trial.suggest_int('max_bin', 32, 100, 255), #450, 550),
            'objective': 'regression',
            
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 150, 256),
            'min_data_in_bin': trial.suggest_int('min_data_in_bin', 1, 150, 256),
            'min_gain_to_split': trial.suggest_discrete_uniform('min_gain_to_split', 0.1, 5, 0.01),
            'metric': 'rmse',
            'learning_rate': trial.suggest_float('learning_rate',0.001,0.3),
            'random_state' : trial.suggest_int('random_state', 0, 42, 1024),
        
            #'min_child_weight': trial.suggest_int('min_child_weight', 1, 16),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 8),
            #'min_child_samples': trial.suggest_int('min_child_samples', 4, 80),
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
            'early_stopping_rounds': 500
            }
    
    # Since we don't use engineered dataset,
    # we need to add feature engineering here
    X_train, X_valid, y_train, y_valid = train_test_split(
        df_train.drop(['id', 'revenue'], axis=1),
        df_train['revenue'],
        test_size=0.2,
        random_state=42
    )
    
    y_train = np.log1p(y_train)
    y_valid = np.log1p(y_valid)
    
    # missing imputation
    cat_imputer_missing = CategoricalImputer(
        imputation_method='missing', variables=CATEGORICAL_VARS_WITH_NA_MISSING)
    X_train = cat_imputer_missing.fit_transform(X_train)
    X_valid = cat_imputer_missing.transform(X_valid)
    
    cat_imputer_frequent = CategoricalImputer(
        imputation_method='frequent', variables=CATEGORICAL_VARS_WITH_NA_FREQUENT)
    X_train = cat_imputer_frequent.fit_transform(X_train)
    X_valid = cat_imputer_frequent.transform(X_valid)
    
    
    # add missing indicator
    missing_indicator = AddMissingIndicator(
        variables=NUMERICAL_VARS_WITH_NA)
    X_train = missing_indicator.fit_transform(X_train)
    X_valid = missing_indicator.transform(X_valid)
    
    
    # impute numerical
    mean_imputation = MeanMedianImputer(
        imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA)
    X_train = mean_imputation.fit_transform(X_train)
    X_valid = mean_imputation.transform(X_valid)
    
    
    # LOG transformation
    log = LogTransformer(variables=NUMERICAL_LOG_VARS)
    X_train = log.fit_transform(X_train)
    X_valid = log.transform(X_valid)
    
    
    # Mappers
    X_train['num_genres'] = X_train['num_genres'].map(genre_mappings)
    X_valid['num_genres'] = X_valid['num_genres'].map(genre_mappings)
    
    X_train['num_pro_comps'] = X_train['num_pro_comps'].map(comps_mappings)
    X_valid['num_pro_comps'] = X_valid['num_pro_comps'].map(comps_mappings)
    
    X_train['num_pro_count'] = X_train['num_pro_count'].map(count_mappings)
    X_valid['num_pro_count'] = X_valid['num_pro_count'].map(count_mappings)
    
    X_train['num_spoken_lang'] = X_train['num_spoken_lang'].map(lang_mappings)
    X_valid['num_spoken_lang'] = X_valid['num_spoken_lang'].map(lang_mappings)
    
    X_train['num_keywords'] = X_train['num_keywords'].map(keywords_mappings)
    X_valid['num_keywords'] = X_valid['num_keywords'].map(keywords_mappings)
    
    X_train['num_crew'] = X_train['num_crew'].map(crew_mappings)
    X_valid['num_crew'] = X_valid['num_crew'].map(crew_mappings)
    
    
    # scaler
    scaler = MinMaxScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train),
                          columns=X_train.columns)
    X_valid = pd.DataFrame(scaler.transform(X_valid),
                          columns=X_train.columns)
    
    
    model = lgb.train(params, 
                      lgb.Dataset(X_train, y_train), 
                      valid_sets=[lgb.Dataset(X_valid, y_valid)], 
                      verbose_eval=100, 
                      num_boost_round=800,
                      early_stopping_rounds=500)
    
    #train_pred = model.predict(X_train)
    valid_pred  = model.predict(X_valid)
    
    #train_score = sqrt(mean_squared_error(y_train, train_pred))
    valid_score = sqrt(mean_squared_error(y_valid, valid_pred))
    
    #print('train_score: ', f"RMSE = {train_score}")
    print('valid_score: ', f"RMSE = {valid_score}")
    
    return valid_score

In [31]:
# set up optuna
# and use RandomSampler
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.RandomSampler())
study.optimize(objective, n_trials=500)

[I 2022-02-23 14:47:58,409] A new study created in memory with name: no-name-1123eb56-1f59-4267-9822-28a9e547084c


Training until validation scores don't improve for 500 rounds
[100]	valid_0's rmse: 2.01114
[200]	valid_0's rmse: 2.02117
[300]	valid_0's rmse: 2.04188
[400]	valid_0's rmse: 2.04796
[500]	valid_0's rmse: 2.04796
[600]	valid_0's rmse: 2.04796


[I 2022-02-23 14:47:58,737] Trial 0 finished with value: 1.9945828467742448 and parameters: {'n_estimators': 10000, 'num_leaves': 13, 'boosting': 'gbdt', 'max_depth': 3, 'max_bin': 32, 'min_data_in_leaf': 1, 'min_data_in_bin': 1, 'min_gain_to_split': 1.5000000000000002, 'learning_rate': 0.1750591182221149, 'random_state': 0, 'feature_fraction': 0.6516799969159521, 'bagging_fraction': 0.8410638440068083, 'bagging_freq': 2, 'lambda_l1': 0.31064770186362567, 'lambda_l2': 9.478326782163505e-06}. Best is trial 0 with value: 1.9945828467742448.


Early stopping, best iteration is:
[165]	valid_0's rmse: 1.99458
valid_score:  RMSE = 1.9945828467742448
Training until validation scores don't improve for 500 rounds
[100]	valid_0's rmse: 2.07833
[200]	valid_0's rmse: 2.02399
[300]	valid_0's rmse: 2.01542


[I 2022-02-23 14:47:59,108] Trial 1 finished with value: 2.012662475978034 and parameters: {'n_estimators': 5000, 'num_leaves': 11, 'boosting': 'gbdt', 'max_depth': 14, 'max_bin': 32, 'min_data_in_leaf': 1, 'min_data_in_bin': 1, 'min_gain_to_split': 4.16, 'learning_rate': 0.0261629072737777, 'random_state': 0, 'feature_fraction': 0.5434541032042388, 'bagging_fraction': 0.9110792915558577, 'bagging_freq': 2, 'lambda_l1': 0.1029044108492442, 'lambda_l2': 1.8468115473889032e-06}. Best is trial 0 with value: 1.9945828467742448.


...

[100]	valid_0's rmse: 2.06017
[200]	valid_0's rmse: 2.06017
[300]	valid_0's rmse: 2.06017
[400]	valid_0's rmse: 2.06017
[500]	valid_0's rmse: 2.06017
Early stopping, best iteration is:
[16]	valid_0's rmse: 2.04427
valid_score:  RMSE = 2.0442683951143774
Training until validation scores don't improve for 500 rounds


[I 2022-02-23 14:50:37,596] Trial 498 finished with value: 2.0036204728533598 and parameters: {'n_estimators': 10000, 'num_leaves': 9, 'boosting': 'gbdt', 'max_depth': 13, 'max_bin': 32, 'min_data_in_leaf': 1, 'min_data_in_bin': 1, 'min_gain_to_split': 1.2000000000000002, 'learning_rate': 0.1615376452998975, 'random_state': 0, 'feature_fraction': 0.5112340556041084, 'bagging_fraction': 0.5730345986878583, 'bagging_freq': 4, 'lambda_l1': 6.107828800529105e-07, 'lambda_l2': 0.0002222471475937367}. Best is trial 354 with value: 1.967147777953415.


[100]	valid_0's rmse: 2.07504
[200]	valid_0's rmse: 2.10727
[300]	valid_0's rmse: 2.11877
[400]	valid_0's rmse: 2.16651
[500]	valid_0's rmse: 2.16739
Early stopping, best iteration is:
[36]	valid_0's rmse: 2.00362
valid_score:  RMSE = 2.0036204728533598
Training until validation scores don't improve for 500 rounds


[I 2022-02-23 14:50:37,823] Trial 499 finished with value: 2.0203090374075807 and parameters: {'n_estimators': 10000, 'num_leaves': 15, 'boosting': 'gbdt', 'max_depth': 5, 'max_bin': 32, 'min_data_in_leaf': 1, 'min_data_in_bin': 1, 'min_gain_to_split': 2.62, 'learning_rate': 0.14964004916437734, 'random_state': 0, 'feature_fraction': 0.9457451256572053, 'bagging_fraction': 0.935665628830114, 'bagging_freq': 4, 'lambda_l1': 2.7149337323559034e-08, 'lambda_l2': 0.003640324086446013}. Best is trial 354 with value: 1.967147777953415.


[100]	valid_0's rmse: 2.0214
[200]	valid_0's rmse: 2.0214
[300]	valid_0's rmse: 2.0214
[400]	valid_0's rmse: 2.0214
[500]	valid_0's rmse: 2.0214
Early stopping, best iteration is:
[51]	valid_0's rmse: 2.02031
valid_score:  RMSE = 2.0203090374075807


In [32]:
# print best parameters we found
study.best_params

{'n_estimators': 10000,
 'num_leaves': 15,
 'boosting': 'gbdt',
 'max_depth': 5,
 'max_bin': 32,
 'min_data_in_leaf': 1,
 'min_data_in_bin': 1,
 'min_gain_to_split': 3.87,
 'learning_rate': 0.13559623067878163,
 'random_state': 0,
 'feature_fraction': 0.4629352923314487,
 'bagging_fraction': 0.8352156581622462,
 'bagging_freq': 7,
 'lambda_l1': 4.339454011245544e-08,
 'lambda_l2': 0.0031662095674507883}

In [33]:
# print best rmse value we found
study.best_value

1.967147777953415

## lgb model

In [34]:
def engineering_data(df_train, df_test):
    X_train, X_valid, y_train, y_valid = train_test_split(
        df_train.drop(['id', 'revenue'], axis=1),
        df_train['revenue'],
        test_size=0.2,
        random_state=42)
    
    X_test = df_test.drop(['id'], axis=1)
    
    # missing imputation
    cat_imputer_missing = CategoricalImputer(
        imputation_method='missing', variables=CATEGORICAL_VARS_WITH_NA_MISSING)
    X_train = cat_imputer_missing.fit_transform(X_train)
    X_valid = cat_imputer_missing.transform(X_valid)
    X_test = cat_imputer_missing.transform(X_test)
    
    cat_imputer_frequent = CategoricalImputer(
        imputation_method='frequent', variables=CATEGORICAL_VARS_WITH_NA_FREQUENT)
    X_train = cat_imputer_frequent.fit_transform(X_train)
    X_valid = cat_imputer_frequent.transform(X_valid)
    X_test = cat_imputer_frequent.transform(X_test)
    
    
    # add missing indicator
    missing_indicator = AddMissingIndicator(
        variables=NUMERICAL_VARS_WITH_NA)
    X_train = missing_indicator.fit_transform(X_train)
    X_valid = missing_indicator.transform(X_valid)
    X_test = missing_indicator.transform(X_test)    

    
    # impute numerical
    mean_imputation = MeanMedianImputer(
            imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA)
    X_train = mean_imputation.fit_transform(X_train)
    X_valid = mean_imputation.transform(X_valid)
    X_test = mean_imputation.transform(X_test)    


    median_test_imputation = MeanMedianImputer(
            imputation_method='median', variables=NEW_VARS_WITH_NA)
    X_train = median_test_imputation.fit_transform(X_train)
    X_test = median_test_imputation.transform(X_test)

    # LOG transformation
    log = LogTransformer(variables=NUMERICAL_LOG_VARS)
    X_train = log.fit_transform(X_train)
    X_valid = log.transform(X_valid)
    X_test = log.transform(X_test)    

    # Mappers
    X_train['num_genres'] = X_train['num_genres'].map(genre_mappings)
    X_valid['num_genres'] = X_valid['num_genres'].map(genre_mappings)
    X_test['num_genres'] = X_test['num_genres'].map(genre_mappings)

    X_train['num_pro_comps'] = X_train['num_pro_comps'].map(comps_mappings)
    X_valid['num_pro_comps'] = X_valid['num_pro_comps'].map(comps_mappings)
    X_test['num_pro_comps'] = X_test['num_pro_comps'].map(comps_mappings)

    X_train['num_pro_count'] = X_train['num_pro_count'].map(count_mappings)
    X_valid['num_pro_count'] = X_valid['num_pro_count'].map(count_mappings)
    X_test['num_pro_count'] = X_test['num_pro_count'].map(count_mappings)    

    X_train['num_spoken_lang'] = X_train['num_spoken_lang'].map(lang_mappings)
    X_valid['num_spoken_lang'] = X_valid['num_spoken_lang'].map(lang_mappings)
    X_test['num_spoken_lang'] = X_test['num_spoken_lang'].map(lang_mappings)

    X_train['num_keywords'] = X_train['num_keywords'].map(keywords_mappings)
    X_valid['num_keywords'] = X_valid['num_keywords'].map(keywords_mappings)
    X_test['num_keywords'] = X_test['num_keywords'].map(keywords_mappings)

    X_train['num_crew'] = X_train['num_crew'].map(crew_mappings)
    X_valid['num_crew'] = X_valid['num_crew'].map(crew_mappings)
    X_test['num_crew'] = X_test['num_crew'].map(crew_mappings)  

    # scaler
    scaler = MinMaxScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train),
                              columns=X_train.columns)
    X_valid = pd.DataFrame(scaler.transform(X_valid),
                              columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test),
                         columns=X_train.columns)    

    return X_train, X_valid, y_train, y_valid, X_test

In [35]:
X_train, X_valid, y_train, y_valid, X_test = engineering_data(df_train, df_test)

In [36]:
X_train.shape, X_valid.shape, X_test.shape

((2400, 20), (600, 20), (4398, 20))

In [37]:
y_train = np.log(y_train)
y_valid = np.log(y_valid)

In [38]:
lgb_model = lgb.LGBMRegressor(**(study.best_params))
lgb_model.fit(X_train, y_train)

LGBMRegressor(bagging_fraction=0.8352156581622462, bagging_freq=7,
              boosting='gbdt', feature_fraction=0.4629352923314487,
              lambda_l1=4.339454011245544e-08, lambda_l2=0.0031662095674507883,
              learning_rate=0.13559623067878163, max_bin=32, max_depth=5,
              min_data_in_bin=1, min_data_in_leaf=1, min_gain_to_split=3.87,
              n_estimators=10000, num_leaves=15, random_state=0)

In [39]:
train_preds = lgb_model.predict(X_train)
sqrt(mean_squared_error(y_train, train_preds))

1.7075463132310413

In [40]:
valid_preds = lgb_model.predict(X_valid)
sqrt(mean_squared_error(y_valid, valid_preds))

1.981864758237628

In [41]:
test_id = df_test['id']

In [42]:
new_preds = pd.Series(np.exp(lgb_model.predict(X_test)), name='revenue')
new_preds = pd.concat([test_id, new_preds], axis=1)
new_preds.head()

,id,revenue
0,3001,6.019024e+05
1,3002,5.953367e+05
2,3003,4.210477e+06
3,3004,1.271916e+07
4,3005,6.397943e+05


In [43]:
# save submission file for kaggle
new_preds.to_csv(path/'sub_lgb4.csv', index=False)

In [44]:
import joblib

In [45]:
# save best parameters for deployment
joblib.dump(study.best_params, 'best_params.joblib')

['best_params.joblib']